# 1 - Clean images & augment them

In [ ]:
import os
from PIL import Image
import imgaug.augmenters as iaa
import numpy as np

source_directory = './pokemon'
target_directory = './pokemon-augmented'

def convert_images_to_jpeg(source_dir, target_dir):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif', 'tiff')):
                file_path = os.path.join(root, file)
                with Image.open(file_path) as img:
                    img = img.convert('RGB')
                    relative_path = os.path.relpath(root, source_dir)
                    target_subdir = os.path.join(target_dir, relative_path)
                    if not os.path.exists(target_subdir):
                        os.makedirs(target_subdir)
                    
                    target_file_path = os.path.join(target_subdir, f"{os.path.splitext(file)[0]}.jpg")
                    
                    img.save(target_file_path, 'JPEG')

def augment_images(directory, num_augmentations=3):
    aug = iaa.Sequential([
        iaa.Fliplr(0.3),  
        iaa.Flipud(0.1), 
        iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},  
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
            rotate=(-25, 25),  
            shear=(-8, 8)  
        ),
        iaa.GaussianBlur(sigma=(0, 1.0)),  
        iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  
        iaa.AdditiveLaplaceNoise(scale=(0, 0.02*255)),  
        iaa.AdditivePoissonNoise(lam=(0, 40)),  
        iaa.Multiply((0.8, 1.2)),  
        iaa.LinearContrast((0.8, 1.2))  
    ])

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('jpg', 'jpeg')):
                file_path = os.path.join(root, file)
                with Image.open(file_path) as img:
                    img_array = np.array(img)

                    for i in range(num_augmentations):
                        # Apply augmentation
                        augmented_image = aug(image=img_array)

                        # Save the augmented image
                        aug_img_pil = Image.fromarray(augmented_image)
                        aug_file_path = os.path.join(root, f"{os.path.splitext(file)[0]}_aug{i}.jpg")
                        aug_img_pil.save(aug_file_path)


In [ ]:
convert_images_to_jpeg(source_directory, target_directory)
augment_images(target_directory)

# 2 - Show how it augmented the images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def display_augmented_images(directory):
    count = 0
    for root, dirs, files in os.walk(directory):
        for dir in dirs[:6]: 
            dir_path = os.path.join(root, dir)
            augmented_images = []
            for file in os.listdir(dir_path):
                if file.lower().endswith(('jpg', 'jpeg')) and 'aug' in file:
                    file_path = os.path.join(dir_path, file)
                    img = Image.open(file_path)
                    augmented_images.append(np.array(img))
                if len(augmented_images) == 3: 
                    break
            
            if augmented_images:
                fig, axs = plt.subplots(1, 3, figsize=(15, 5))
                for ax, img in zip(axs, augmented_images):
                    ax.imshow(img)
                    ax.axis('off')
                plt.show()
                count += 1
            if count >= 6:
                return

In [ ]:
display_augmented_images(target_directory)

# 3 - Resize images

In [1]:
from PIL import Image
import os

def ensure_jpg_extension(output_path):
    root, ext = os.path.splitext(output_path)
    if ext.lower() != '.jpg':
        output_path = root + '.jpg'
    return output_path

def resize_image(input_path, output_path, size=(224, 224), quality=85):
    with Image.open(input_path) as img:
        if img.size[0] < size[0] or img.size[1] < size[1]:
            print(f"Ignored {input_path}: smaller than resize dimensions {size}.")
            return
        img = img.convert('RGB')
        img = img.resize(size, Image.LANCZOS)
        img.save(ensure_jpg_extension(output_path), optimize=True, quality=quality)

def resize_images_in_directory(input_directory, output_directory, size=(224, 224), quality=85):
    for root, dirs, files in os.walk(input_directory):
        for filename in files:
            if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                input_path = os.path.join(root, filename)
                relative_path = os.path.relpath(root, input_directory)
                output_folder = os.path.join(output_directory, relative_path)
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)
                output_path = os.path.join(output_folder, filename)
                resize_image(input_path, output_path, size, quality)

input_directory = './pokemon'
output_directory = './pokemon-resized'
resize_images_in_directory(input_directory, output_directory, (192, 192), quality=90)

Ignored ./pokemon/Zapdos/00e5ca5e91ca4f44be3d3d2383f96779.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Zapdos/1d26634c068d43c6a994340a85ec79a6.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Zapdos/103cea46936b43018c685f8b9f8e1e11.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Zapdos/c0d9680eeb85459f93205ba568c0bd8f.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Zapdos/34f5854ee25f415481e5ef1142baf632.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Kadabra/f23d67da38ec46e7aaadc3aacedec8fa.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Kadabra/deaab18406cd47939188016442384216.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Kadabra/463e4d45c0514d4e8a8974004fc14dd6.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Kadabra/f02640d4ea714d91ba16fafd46b3a49b.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Kadabra/ef3fdd8c18ca4aff

/opt/homebrew/Caskroom/miniconda/base/envs/tfml/lib/python3.11/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Ignored ./pokemon/Jolteon/2e1f03093efa4f4683761505b0ab4863.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Jolteon/6e82df59b14e4a7caea9ac2d0c194699.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Jolteon/96ec94e0e05f4e73adaf2b2ff68cb427.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Jolteon/e4884100758042c2a11a2dcf6122fde7.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Jolteon/cb4cedeed46d44cbb54c75ed55bcf67a.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Jolteon/bd478b3dde5c4050bc2cfa974739d55f.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Jolteon/6acf956e971846209b3eb02407d2a3d4.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Jolteon/5858ad1fd64c44228411411d4c25cecf.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Jolteon/51b75c05f63c499491c8f1bdda54558e.jpg: smaller than resize dimensions (192, 192).
Ignored ./pokemon/Jolteon/4d8fc9f19b7